In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-05-30T10:17:36.823019-07:00

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.13.2

Compiler    : GCC 11.3.0
OS          : Linux
Release     : 5.15.0-1017-nvidia
Machine     : x86_64
Processor   : x86_64
CPU cores   : 224
Architecture: 64bit



In [3]:
%watermark --gpu

GPU Info: 
  GPU 0: NVIDIA H100 80GB HBM3
  GPU 1: NVIDIA H100 80GB HBM3
  GPU 2: NVIDIA H100 80GB HBM3
  GPU 3: NVIDIA H100 80GB HBM3
  GPU 4: NVIDIA H100 80GB HBM3
  GPU 5: NVIDIA H100 80GB HBM3
  GPU 6: NVIDIA H100 80GB HBM3
  GPU 7: NVIDIA H100 80GB HBM3



In [4]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
from dask.delayed import delayed
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
import logging

/home/dgxuser/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
%watermark --iversions

optuna : 3.1.1
xgboost: 1.7.5
logging: 0.5.1.2
numpy  : 1.23.5
pandas : 1.5.3
dask   : 2023.3.2



In [6]:
cluster = LocalCUDACluster(n_workers=8)
client = Client(cluster)

2023-05-30 10:17:45,953 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-30 10:17:45,953 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-30 10:17:45,953 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-30 10:17:45,953 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-30 10:17:45,956 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-30 10:17:45,956 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-30 10:17:45,957 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-30 10:17:45,957 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-30 10:17:45,960 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-30 10:17:45,960 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-

In [7]:
optuna_log = pd.read_csv('optuna_xgb_output_l_4.csv')

In [8]:
optuna_log.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_gamma,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,1153.904710,0.373965,0.980291,rmse,2.497604,0.014629,0.007077,21,103,0.583175,COMPLETE
1,1,1152.803715,0.004224,0.856359,rmse,0.004733,3.143904,0.041572,11,39,0.653761,COMPLETE
2,2,1152.649488,1.043722,0.903962,rmse,0.709577,3.394003,0.012027,6,49,0.431573,COMPLETE
3,3,1200.088834,0.056953,0.339802,rmse,0.002243,0.008525,0.073172,16,45,0.594994,COMPLETE
4,4,1255.695543,0.246479,0.914929,rmse,0.445993,0.173519,0.002016,25,255,0.547362,COMPLETE


In [9]:
optuna_log.value.min()

1135.212585472503

In [10]:
study = optuna.load_study(storage="sqlite:///xgb_optuna_allstate_l_4.db", study_name="five_fold_optuna_xgb_l_4")

In [11]:
study.best_params

{'alpha': 1.8620662483899058,
 'colsample_bytree': 0.47780377425479476,
 'eval_metric': 'rmse',
 'gamma': 0.0016435353992643332,
 'lambda': 0.40986852781746064,
 'learning_rate': 0.01805814216145912,
 'max_depth': 15,
 'min_child_weight': 204,
 'subsample': 0.9847422024810327}

In [12]:
num_round = 1000

params = study.best_params
params['objective'] ='reg:squarederror'
params['base_score'] = 7.76
params['tree_method'] ='gpu_hist'
params['eval_metric'] ='rmse'

In [33]:
train = pd.read_csv('../input/train.csv.zip')
x_train_l = pd.read_csv('../input/x_train_l.csv')
x_test_l = pd.read_csv(f'../input/x_test_l.csv')
x_test_l.to_csv('../input/x_test_l.csv.zip', compression='zip', index=False)
x_train_l['loss'] = np.log(train.loss.values + 200)
x_train_l.to_csv('../input/x_train_l.csv.zip', compression='zip')

In [60]:
x_train_l = dd.read_csv(f'../input/x_train_l.csv.zip')
x_test_l = dd.read_csv(f'../input/x_test_l.csv.zip')
train_y = x_train_l['loss']
x_train_l = x_train_l[x_train_l.columns.difference(['loss'])]

/home/dgxuser/.local/lib/python3.10/site-packages/dask/dataframe/io/csv.py:542: UserWarning: Warning zip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  warn(
/home/dgxuser/.local/lib/python3.10/site-packages/dask/dataframe/io/csv.py:542: UserWarning: Warning zip compression does not support breaking apart files
Please ensure that each individual file can fit in memory and
use the keyword ``blocksize=None to remove this message``
Setting ``blocksize=None``
  warn(


In [61]:
x_train_l.columns

Index(['cat1', 'cat10', 'cat100', 'cat101', 'cat102', 'cat103', 'cat104',
       'cat105', 'cat106', 'cat107',
       ...
       'cont13', 'cont14', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6',
       'cont7', 'cont8', 'cont9'],
      dtype='object', length=130)

In [62]:
len(x_train_l.columns)

130

In [63]:
len(x_test_l.columns)

130

In [59]:
x_test_l.columns

Index(['Unnamed: 0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
       'cat8', 'cat9',
       ...
       'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11',
       'cont12', 'cont13', 'cont14'],
      dtype='object', length=131)

In [64]:
dtrain = xgb.dask.DaskDMatrix(client, x_train_l.values, train_y, enable_categorical=True)
dtest = xgb.dask.DaskDMatrix(client, x_test_l.values,  enable_categorical=True)
        


In [65]:
%%time
output = xgb.dask.train(client, params, dtrain, num_round)


INFO:distributed.worker:Run out-of-band function '_start_tracker'
INFO:distributed.scheduler:Receive client connection: Client-worker-65d13c3d-ff12-11ed-a850-b5adb1c07716
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:53976
[10:50:00] task [xgboost.dask-6]:tcp://127.0.0.1:46767 got new rank 0


CPU times: user 2.14 s, sys: 636 ms, total: 2.78 s
Wall time: 9.02 s


In [66]:
%%time
booster = output['booster']  # booster is the trained model
booster.set_param({'predictor': 'gpu_predictor'})
predictions = xgb.dask.predict(client, booster, dtest)
predictions = predictions.compute()

CPU times: user 124 ms, sys: 98.4 ms, total: 222 ms
Wall time: 1.44 s


In [67]:
(np.exp(predictions)-200)[:10]

array([2614.168 , 3541.6987, 3216.981 , 2625.9463, 1475.0652, 2244.0916,
       2405.0142, 1757.0818, 2359.3616, 2153.6968], dtype=float32)

In [68]:
sample_submission = pd.read_csv('../input/sample_submission.csv.zip')

In [69]:
sample_submission['loss'] = (np.exp(predictions)-200)
sample_submission.head()

,id,loss
0,4,2614.167969
1,6,3541.698730
2,9,3216.980957
3,12,2625.946289
4,15,1475.065186


In [70]:
sample_submission.to_csv('../submissions/new_sub.csv.gz', index=False, compression='gzip')